# Instruction 확인하는 코드

In [3]:
import re
import os
from datasets import load_from_disk

# ==============================================================================
# 1. 경로 설정 (요청하신 대로 매핑)
# ==============================================================================
# Writer (Testset)
path_test_set = "Mol-LLM_Custom/checkpoint/mol-llm_testset"

# Download (My Dataset)
path_my_dataset = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace"

def get_clean_instruction(prompt_text):
    """
    prompt_text에서 <SELFIES> ... </GRAPH> 구간을 제거하여
    순수 Instruction(템플릿)만 추출합니다.
    """
    if not prompt_text:
        return ""
        
    # <SELFIES> 로 시작해서 </GRAPH> 로 끝나는 패턴 제거 (DOTALL: 줄바꿈 포함)
    pattern = r"<SELFIES>.*?</GRAPH>"
    
    # 해당 패턴을 placeholder로 치환
    clean_text = re.sub(pattern, "<INPUT_MOLECULE_PLACEHOLDER>", prompt_text, flags=re.DOTALL)
    
    return clean_text.strip()

def compare_bace_instructions(dataset_path_writer, dataset_path_my, task_name="bace"):
    print(f"🔍 [Comparison Start] Task: {task_name}")
    print(f"   path_test_set (Writer): {dataset_path_writer}")
    print(f"   path_my_dataset (Download): {dataset_path_my}\n")

    # 1. 데이터셋 로드
    try:
        ds_writer = load_from_disk(dataset_path_writer)
        ds_my = load_from_disk(dataset_path_my)
    except Exception as e:
        print(f"❌ Error loading datasets: {e}")
        return

    # 2. BACE 태스크 필터링 함수 (대소문자 무시 및 여러 컬럼 확인)
    def filter_func(x):
        # 1. 'task'나 'dataset' 컬럼 확인
        for col in ['task', 'task_subtask_pair', 'dataset']:
            if x.get(col) and task_name in str(x[col]).lower():
                return True
        # 2. 컬럼이 없으면 텍스트 내용 확인 (fallback)
        text_content = x.get('prompt_text') or x.get('text') or x.get('instruction') or ""
        if task_name in str(text_content).lower():
            return True
        return False

    print("⏳ Filtering 'bace' task...")
    ds_writer_bace = ds_writer.filter(filter_func)
    ds_my_bace = ds_my.filter(filter_func)

    print(f"📊 [Count Check]")
    print(f"   Writer (Testset) '{task_name}' samples: {len(ds_writer_bace)}")
    print(f"   My (Download)    '{task_name}' samples: {len(ds_my_bace)}")

    if len(ds_writer_bace) == 0 or len(ds_my_bace) == 0:
        print("⚠️ Warning: One of the datasets has 0 samples for this task. Check filter logic.")
        return

    # 3. Instruction 추출 및 집합(Set) 생성
    instructions_writer = set()
    instructions_my = set()

    # 텍스트가 들어있는 컬럼명 찾기 함수
    def get_text_column(ds):
        for candidate in ['prompt_text', 'text', 'instruction', 'input']:
            if candidate in ds.column_names:
                return candidate
        return None

    col_writer = get_text_column(ds_writer_bace)
    col_my = get_text_column(ds_my_bace)

    if not col_writer or not col_my:
        print(f"❌ Could not find text column. Available keys: {ds_writer_bace.column_names} / {ds_my_bace.column_names}")
        return

    # Writer 데이터셋 추출
    for i, item in enumerate(ds_writer_bace):
        clean_inst = get_clean_instruction(item[col_writer])
        instructions_writer.add(clean_inst)
        if i == 0:
            print(f"\n📝 [Example Instruction from Writer Dataset]")
            print(f"{clean_inst}")

    # My 데이터셋 추출
    for item in ds_my_bace:
        clean_inst = get_clean_instruction(item[col_my])
        instructions_my.add(clean_inst)

    # 4. 비교 분석
    common_inst = instructions_writer.intersection(instructions_my)
    only_in_writer = instructions_writer - instructions_my
    only_in_my = instructions_my - instructions_writer

    print(f"\n⚖️  [Analysis Result]")
    print(f"   Unique Instructions in Writer: {len(instructions_writer)}")
    print(f"   Unique Instructions in My Data: {len(instructions_my)}")
    print(f"   Common Instructions (Match):    {len(common_inst)}")

    if len(only_in_writer) == 0 and len(only_in_my) == 0:
        print("\n✅ SUCCESS: The instruction sets are IDENTICAL!")
    else:
        print("\n❌ DIFFERENCE DETECTED!")
        
        if len(only_in_writer) > 0:
            print(f"\n[!] Instructions ONLY in Writer Dataset ({len(only_in_writer)}):")
            for idx, inst in enumerate(list(only_in_writer)[:3]):
                print(f"   {idx+1}. {inst[:100]}...")

        if len(only_in_my) > 0:
            print(f"\n[!] Instructions ONLY in My Dataset ({len(only_in_my)}):")
            for idx, inst in enumerate(list(only_in_my)[:3]):
                print(f"   {idx+1}. {inst[:100]}...")

# ==============================================================================
# 실행
# ==============================================================================
compare_bace_instructions(path_test_set, path_my_dataset, task_name="bace")

🔍 [Comparison Start] Task: bace
   path_test_set (Writer): Mol-LLM_Custom/checkpoint/mol-llm_testset
   path_my_dataset (Download): Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace

⏳ Filtering 'bace' task...


Filter:   0%|          | 0/58757 [00:00<?, ? examples/s]

Filter: 100%|██████████| 152/152 [00:00<00:00, 1981.65 examples/s]


📊 [Count Check]
   Writer (Testset) 'bace' samples: 152
   My (Download)    'bace' samples: 152

📝 [Example Instruction from Writer Dataset]
<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. 

Using <INPUT_MOLECULE_PLACEHOLDER> as the molecule, tell me the biological activity against BACE-1. [/INST]

⚖️  [Analysis Result]
   Unique Instructions in Writer: 13
   Unique Instructions in My Data: 13
   Common Instructions (Match):    13

✅ SUCCESS: The instruction sets are IDENTICAL!


# Moelcular 종류와 graph feature 확인하는 코드

In [2]:
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm
import torch

# ==========================================
# 1. 설정: 경로 입력
# ==========================================
dataset_download_InstructGraph = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace"
dataset_writer = "Mol-LLM_Custom/checkpoint/mol-llm_testset"

def deep_compare_datasets(path1, path2):
    print(f"📂 Loading Dataset 1 (Reference): {path1}")
    ds1 = load_from_disk(path1)
    
    print(f"📂 Loading Dataset 2 (Target): {path2}")
    ds2_raw = load_from_disk(path2)
    print(f"   Original Dataset 2 size: {len(ds2_raw)}")

    # ==========================================
    # 2. Dataset 2에서 'bace' 필터링 수행
    # ==========================================
    print("🔍 Filtering 'bace' from Dataset 2...")
    
    # 필터링 함수 정의
    def filter_bace(example):
        # 1순위: 'task'나 'dataset' 컬럼이 있으면 확인
        if 'task' in example and example['task'] is not None:
            if 'bace' in str(example['task']).lower():
                return True
        if 'dataset' in example and example['dataset'] is not None:
            if 'bace' in str(example['dataset']).lower():
                return True
        
        # 2순위: 컬럼이 없으면 Instruction(지시문) 텍스트 안에 'bace'가 있는지 확인
        if 'instruction' in example:
            if 'bace' in example['instruction'].lower():
                return True
        
        return False

    # 필터 적용
    ds2 = ds2_raw.filter(filter_bace)
    print(f"   Filtered Dataset 2 size: {len(ds2)} (Only 'bace')")

    # ==========================================
    # 3. 데이터 개수 및 정렬 확인
    # ==========================================
    if len(ds1) != len(ds2):
        print(f"❌ SIZE MISMATCH: Dataset 1 has {len(ds1)}, but Filtered Dataset 2 has {len(ds2)}.")
        print("   -> 개수가 다르면 1:1 비교가 불가능합니다. 필터링 조건을 확인해주세요.")
        return

    print(f"✅ Sizes match: {len(ds1)} samples. Starting deep comparison...")

    mismatch_count = 0
    keys_to_check = ['selfies', 'graph', 'input_ids', 'labels'] 
    
    # tqdm으로 진행 상황 표시
    for idx in tqdm(range(len(ds1)), desc="Comparing Samples"):
        sample1 = ds1[idx]
        sample2 = ds2[idx]

        diff_found = False
        log_msg = []

        # === 4. SELFIES (문자열) 비교 ===
        # 키 이름이 다를 수 있으니 확인 (selfies, input, text 등)
        key_s1 = 'selfies' if 'selfies' in sample1 else 'input'
        key_s2 = 'selfies' if 'selfies' in sample2 else 'input'
        
        # 양쪽에 해당 키가 있을 때만 비교
        if key_s1 in sample1 and key_s2 in sample2:
            val1 = sample1[key_s1]
            val2 = sample2[key_s2]
            if val1 != val2:
                diff_found = True
                log_msg.append(f"String mismatch: ds1['{key_s1}'] != ds2['{key_s2}']")
                # 내용이 너무 길 수 있으니 앞부분만 출력
                log_msg.append(f"   -> 1: {str(val1)[:30]}...")
                log_msg.append(f"   -> 2: {str(val2)[:30]}...")

        # === 5. Graph Data (텐서/배열) 비교 ===
        if 'graph' in sample1 and 'graph' in sample2:
            g1 = sample1['graph']
            g2 = sample2['graph']
            
            # (1) Node Features (x)
            k1 = 'x' if 'x' in g1 else 'node_feat'
            k2 = 'x' if 'x' in g2 else 'node_feat'
            
            if k1 in g1 and k2 in g2:
                arr1 = np.array(g1[k1])
                arr2 = np.array(g2[k2])
                if arr1.shape != arr2.shape:
                    diff_found = True
                    log_msg.append(f"Graph Node Feature Shape mismatch: {arr1.shape} vs {arr2.shape}")
                elif not np.allclose(arr1, arr2, atol=1e-5): # 소수점 오차 허용
                    diff_found = True
                    log_msg.append(f"Graph Node Feature ({k1}) values mismatch")

            # (2) Edge Index
            if 'edge_index' in g1 and 'edge_index' in g2:
                arr1 = np.array(g1['edge_index'])
                arr2 = np.array(g2['edge_index'])
                if arr1.shape != arr2.shape:
                    diff_found = True
                    log_msg.append(f"Edge Index Shape mismatch: {arr1.shape} vs {arr2.shape}")
                elif not np.array_equal(arr1, arr2): # 정수는 완전 일치
                    diff_found = True
                    log_msg.append("Graph Edge Index mismatch (Different connections)")

            # (3) Edge Attributes
            if 'edge_attr' in g1 and 'edge_attr' in g2:
                arr1 = np.array(g1['edge_attr'])
                arr2 = np.array(g2['edge_attr'])
                if not np.allclose(arr1, arr2, atol=1e-5):
                    diff_found = True
                    log_msg.append("Graph Edge Attributes mismatch")

        # 결과 출력
        if diff_found:
            mismatch_count += 1
            print(f"\n⚠️ Mismatch found at Index {idx}:")
            for msg in log_msg:
                print(f"   - {msg}")
            
            if mismatch_count > 5:
                print("\n⛔ Too many mismatches found. Stopping early.")
                break

    if mismatch_count == 0:
        print("\n🎉 SUCCESS: All Filtered BACE samples (Strings & Graphs) are IDENTICAL!")
    else:
        print(f"\n❌ FOUND {mismatch_count} mismatches.")

# 실행
if __name__ == "__main__":
    deep_compare_datasets(dataset_download_InstructGraph, dataset_writer)

📂 Loading Dataset 1 (Reference): Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace
📂 Loading Dataset 2 (Target): Mol-LLM_Custom/checkpoint/mol-llm_testset
   Original Dataset 2 size: 58757
🔍 Filtering 'bace' from Dataset 2...


Filter:   0%|          | 0/58757 [00:00<?, ? examples/s]

Filter: 100%|██████████| 58757/58757 [00:32<00:00, 1821.40 examples/s]


   Filtered Dataset 2 size: 152 (Only 'bace')
✅ Sizes match: 152 samples. Starting deep comparison...


Comparing Samples: 100%|██████████| 152/152 [00:00<00:00, 890.51it/s]


🎉 SUCCESS: All Filtered BACE samples (Strings & Graphs) are IDENTICAL!


# download dataset을 writer의 instruction set으로 변경하는 코드

In [4]:
import re
from datasets import load_from_disk
import os

# ==============================================================================
# 1. 경로 설정
# ==============================================================================
# [Reference] 올바른 형식을 가진 Test Set 경로
path_reference = "Mol-LLM_Custom/checkpoint/mol-llm_testset"

# [Target] 수정하고 싶은 내 데이터셋 경로 (Download)
path_target_original = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace"

# [Output] 수정 후 저장할 새로운 경로
path_output_fixed = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace_FIXED"

# ==============================================================================
# 2. 함수 정의
# ==============================================================================

def extract_template_parts(prompt_text):
    """
    Test Set의 샘플 텍스트에서 분자 정보(<SELFIES>...<GRAPH>...)를 기준으로
    '앞부분(Prefix)'과 '뒷부분(Suffix)'을 분리합니다.
    """
    # 정규표현식: <SELFIES>로 시작해서 </GRAPH>로 끝나는 구간을 찾음 (줄바꿈 포함)
    pattern = r"(.*)(<SELFIES>.*?</GRAPH>)(.*)"
    match = re.search(pattern, prompt_text, re.DOTALL)
    
    if match:
        prefix = match.group(1) # 지시문 앞부분 (<s>[INST] ...)
        # middle = match.group(2) # 분자 정보 (이건 데이터마다 다르므로 버림)
        suffix = match.group(3) # 지시문 뒷부분 ( [/INST] 등)
        return prefix, suffix
    else:
        raise ValueError("❌ 텍스트에서 <SELFIES>...<GRAPH> 구조를 찾을 수 없습니다. 포맷이 다릅니다.")

def extract_molecule_part(prompt_text):
    """
    Target 데이터셋의 텍스트에서 '분자 정보'만 쏙 뽑아냅니다.
    """
    pattern = r"(<SELFIES>.*?</GRAPH>)"
    match = re.search(pattern, prompt_text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        # 만약 텍스트에 태그가 없다면, 직접 구성해야 할 수도 있음 (여기서는 에러 처리)
        return None

# ==============================================================================
# 3. 메인 로직
# ==============================================================================

def standardize_dataset():
    print("📂 Loading Reference Dataset (Test Set)...")
    ds_ref = load_from_disk(path_reference)
    
    # BACE 태스크 샘플 하나만 찾아서 템플릿 추출
    # (효율을 위해 전체를 돌지 않고, bace가 포함된 첫 번째 샘플을 찾음)
    ref_sample = None
    for item in ds_ref:
        t = item.get('prompt_text', '')
        if 'bace' in t.lower() and '<SELFIES>' in t:
            ref_sample = item
            break
            
    if ref_sample is None:
        print("❌ Reference Dataset에서 BACE 관련 샘플을 찾을 수 없습니다.")
        return

    print("🧩 Extracting Template from Reference...")
    try:
        prefix_template, suffix_template = extract_template_parts(ref_sample['prompt_text'])
        print(f"   ✅ Template Extracted successfully!")
        print(f"      Prefix end: ...{prefix_template[-30:]!r}")
        print(f"      Suffix start: {suffix_template[:30]!r}...")
    except ValueError as e:
        print(e)
        return

    print("\n📂 Loading Target Dataset (To be fixed)...")
    ds_target = load_from_disk(path_target_original)
    
    # Target 데이터셋에서 BACE만 필터링 (필요시)
    # ds_target = ds_target.filter(lambda x: 'bace' in str(x.get('task', '')).lower())

    print(f"   Target size: {len(ds_target)}")

    # ============================================================
    # 4. 데이터셋 변환 (Map 함수 적용)
    # ============================================================
    def apply_template(example):
        # 1. 현재 샘플의 원본 텍스트 가져오기
        original_text = example.get('prompt_text') or example.get('text')
        
        # 2. 내 데이터의 '분자 정보' (<SELFIES>...<GRAPH>...) 추출
        molecule_part = extract_molecule_part(original_text)
        
        # 만약 텍스트에서 추출이 안 되면, input_ids나 graph 필드에서 재구성해야 함
        # 여기서는 텍스트가 존재한다고 가정
        if molecule_part is None:
            # 에러 방지를 위해 원본 반환 (혹은 로깅)
            return example 
        
        # 3. Reference 템플릿 + 내 분자 정보 결합
        new_text = prefix_template + molecule_part + suffix_template
        
        # 4. 컬럼 업데이트
        example['prompt_text'] = new_text
        
        # 주의: input_ids가 미리 토큰화되어 저장된 경우, 텍스트가 바뀌었으므로
        # input_ids는 무효화되거나 다시 토크나이징 해야 함.
        # 여기서는 텍스트만 맞추는 것이므로 input_ids는 그대로 두거나 삭제할 수 있음.
        # example['input_ids'] = ... (토크나이저가 있다면 여기서 수행)
        
        return example

    print("\n🔄 Applying Reference Template to Target Dataset...")
    ds_fixed = ds_target.map(apply_template, desc="Standardizing Instructions")

    # ============================================================
    # 5. 결과 확인 및 저장
    # ============================================================
    print("\n📝 Checking Result (First Sample):")
    print(ds_fixed[0]['prompt_text'][:200] + " ... ")

    print(f"\n💾 Saving fixed dataset to: {path_output_fixed}")
    ds_fixed.save_to_disk(path_output_fixed)
    print("✅ Done! 이제 이 데이터셋은 Test Set과 완벽히 동일한 Instruction 형식을 가집니다.")

if __name__ == "__main__":
    standardize_dataset()

📂 Loading Reference Dataset (Test Set)...
🧩 Extracting Template from Reference...
   ✅ Template Extracted successfully!
      Prefix end: ...' molecule generation. \n\nUsing '
      Suffix start: ' as the molecule, tell me the '...

📂 Loading Target Dataset (To be fixed)...
   Target size: 152

🔄 Applying Reference Template to Target Dataset...


Standardizing Instructions: 100%|██████████| 152/152 [00:00<00:00, 5322.77 examples/s]



📝 Checking Result (First Sample):
<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule capti ... 

💾 Saving fixed dataset to: Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace_FIXED


Saving the dataset (1/1 shards): 100%|██████████| 152/152 [00:00<00:00, 15286.39 examples/s]

✅ Done! 이제 이 데이터셋은 Test Set과 완벽히 동일한 Instruction 형식을 가집니다.


# Writer의 Dataset을 전체 Instruction Set 내에서 Random Instruction으로 수정.

In [12]:
import re
import random
import os
from datasets import load_from_disk
from tqdm import tqdm

# ==============================================================================
# 1. 경로 설정
# ==============================================================================

# [Reference] Writer의 전체 데이터셋 (여기서 BACE Instruction만 추출합니다)
path_reference = "Mol-LLM_Custom/checkpoint/mol-llm_testset"

# [Target] 내 BACE 전용 데이터셋 (수정 대상)
path_target = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_bace"

# [Output] 결과 저장 경로
path_output = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_bace_SWAPPED"

# ==============================================================================
# 2. 헬퍼 함수 정의
# ==============================================================================

def extract_parts(text):
    """
    텍스트를 [Instruction 앞부분] + [분자 정보] + [Instruction 뒷부분]으로 분리합니다.
    분자 정보는 <SELFIES> 태그로 시작해서 </GRAPH> 태그로 끝난다고 가정합니다.
    """
    pattern = r"(.*)(<SELFIES>.*?</GRAPH>)(.*)"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        prefix = match.group(1)
        molecule = match.group(2)
        suffix = match.group(3)
        return prefix, molecule, suffix
    return None, None, None

# ==============================================================================
# 3. 메인 로직
# ==============================================================================

def main():
    # --------------------------------------------------------------------------
    # Step 1. Reference 데이터셋에서 BACE Instruction 수집 (Template Pool 생성)
    # --------------------------------------------------------------------------
    print(f"📂 Loading Reference Dataset: {path_reference}")
    ds_ref = load_from_disk(path_reference)
    
    print("🔍 Filtering Reference for 'bace' task...")
    # task 컬럼이 'bace'를 포함하거나, 텍스트 내에 'bace' 관련 내용이 있는지 확인 (대소문자 무시)
    ds_ref_bace = ds_ref.filter(
        lambda x: 'bace' in str(x.get('task', '')).lower()
    )
    
    print(f"   Original Size: {len(ds_ref)} -> BACE Only Size: {len(ds_ref_bace)}")
    
    if len(ds_ref_bace) == 0:
        print("❌ Error: Reference 데이터셋에서 'bace' 태스크를 찾을 수 없습니다.")
        return

    # 템플릿(질문 패턴) 수집
    template_pool = set()
    for item in tqdm(ds_ref_bace, desc="Collecting Templates"):
        text = item.get('prompt_text') or item.get('text', '')
        if '<SELFIES>' not in text: 
            continue
            
        prefix, _, suffix = extract_parts(text)
        if prefix is not None:
            template_pool.add((prefix, suffix))
    
    template_pool = list(template_pool)
    print(f"✅ Collected {len(template_pool)} unique BACE instruction templates.")
    
    if len(template_pool) < 2:
        print("⚠️ Warning: 수집된 템플릿이 1개 이하입니다. 교체 효과가 거의 없을 수 있습니다.")

    # --------------------------------------------------------------------------
    # Step 2. Target 데이터셋 로드 및 Instruction 교체
    # --------------------------------------------------------------------------
    print(f"\n📂 Loading Target Dataset: {path_target}")
    ds_target = load_from_disk(path_target)
    print(f"   Target Size: {len(ds_target)}")

    def swap_instruction_fn(example):
        original_text = example.get('prompt_text') or example.get('text')
        
        # 1. 분자 정보 추출
        curr_prefix, molecule_part, curr_suffix = extract_parts(original_text)
        
        # 분자 정보가 없으면 그대로 반환
        if molecule_part is None:
            return example
        
        # 2. 템플릿 선택 (현재 것과 다른 것을 우선 선택)
        candidates = [t for t in template_pool if t != (curr_prefix, curr_suffix)]
        
        # 만약 후보가 없으면(전부 똑같거나 풀이 작으면) 전체 풀에서 선택
        if not candidates:
            candidates = template_pool
            
        new_prefix, new_suffix = random.choice(candidates)
        
        # 3. 새로운 텍스트 조립
        new_text = new_prefix + molecule_part + new_suffix
        example['prompt_text'] = new_text
        
        # 4. [중요] 텍스트가 바뀌었으므로 기존 토큰 ID(input_ids) 삭제
        #    이렇게 해야 나중에 Dataset 로드 시 텍스트만 보고 새로 토크나이징을 하거나
        #    최소한 예전 토큰을 쓰는 실수를 막을 수 있음.
        if 'input_ids' in example:
            example['input_ids'] = []  # 빈 리스트로 초기화 또는 del example['input_ids']
        if 'attention_mask' in example:
            example['attention_mask'] = []

        return example

    print("\n🔀 Swapping instructions...")
    ds_swapped = ds_target.map(swap_instruction_fn, desc="Processing")

    # --------------------------------------------------------------------------
    # Step 3. 결과 확인 및 저장
    # --------------------------------------------------------------------------
    print("\n📝 Result Preview (First Sample):")
    print("-" * 50)
    print(ds_swapped[0]['prompt_text'][:300] + " ... ")
    print("-" * 50)

    print(f"\n💾 Saving to: {path_output}")
    ds_swapped.save_to_disk(path_output)
    print("✅ Completed Successfully!")

if __name__ == "__main__":
    main()

📂 Loading Reference Dataset: Mol-LLM_Custom/checkpoint/mol-llm_testset
🔍 Filtering Reference for 'bace' task...


Filter:   0%|          | 0/58757 [00:00<?, ? examples/s]

Filter: 100%|██████████| 58757/58757 [00:33<00:00, 1773.71 examples/s]


   Original Size: 58757 -> BACE Only Size: 152


✅ Collected 13 unique BACE instruction templates.

📂 Loading Target Dataset: Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_bace
   Target Size: 152

🔀 Swapping instructions...


Processing: 100%|██████████| 152/152 [00:00<00:00, 4959.19 examples/s]



📝 Result Preview (First Sample):
--------------------------------------------------
<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. 

<SELFIES> [C][C][=C][C][=C][C][=C][Ring1][=Branch1][C][=C][C][=C][N][= ... 
--------------------------------------------------

💾 Saving to: Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_bace_SWAPPED


Saving the dataset (1/1 shards): 100%|██████████| 152/152 [00:00<00:00, 14841.22 examples/s]

✅ Completed Successfully!


# molecule 끼리의 비교

In [2]:
import os
import re
import torch
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# ==============================================================================
# 1. 경로 설정
# ==============================================================================
# [Writer] Reference Dataset (원본)
path_ref = "Mol-LLM_Custom/checkpoint/mol-llm_testset"

# [Target] 비교할 내 데이터셋 (Download 된 것)
path_target = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_blank_bace"

# ==============================================================================
# 2. 유틸리티 함수 (한 줄 출력용)
# ==============================================================================

def format_oneline(value):
    """
    Tensor, List, Dict 등 어떤 값이 오더라도 강제로 줄바꿈을 없애고
    한 줄짜리 문자열로 변환합니다.
    """
    if value is None:
        return "None"
    
    # Tensor나 Array를 리스트로 변환
    if isinstance(value, torch.Tensor):
        value = value.tolist()
    elif isinstance(value, np.ndarray):
        value = value.tolist()
        
    # 문자열로 변환
    s = str(value)
    
    # 1. 줄바꿈(\n) 제거
    s = s.replace('\n', '')
    s = s.replace('\r', '')
    
    # 2. 탭(\t) 제거
    s = s.replace('\t', ' ')
    
    # 3. 연속된 공백을 하나로 줄임
    s = re.sub(r'\s+', ' ', s)
    
    return s.strip()

def get_graph_feature(item, feature_name):
    """
    item['graph'] 안에 있는 feature_name (예: x, edge_index)을 안전하게 꺼냅니다.
    """
    graph_data = item.get('graph')
    if graph_data is None:
        return None

    # graph가 딕셔너리인 경우
    if isinstance(graph_data, dict):
        return graph_data.get(feature_name)
    
    # graph가 PyG Data 객체인 경우 (속성으로 접근)
    if hasattr(graph_data, feature_name):
        return getattr(graph_data, feature_name)
        
    return None

# ==============================================================================
# 3. 메인 비교 로직
# ==============================================================================

def main():
    print("📂 Loading Datasets...")
    ds_ref = load_from_disk(path_ref)
    ds_target = load_from_disk(path_target)
    
    print(f"   Ref Size: {len(ds_ref)}")
    print(f"   Target Size: {len(ds_target)}")

    # -------------------------------------------------------
    # 1. Reference Indexing (Key = prompt_text)
    # -------------------------------------------------------
    print("\n🔍 Indexing Reference Dataset by 'prompt_text'...")
    ref_lookup = {}
    
    # 속도를 위해 BACE 관련 데이터만 필터링하여 인덱싱 (선택사항)
    ds_ref_filtered = ds_ref.filter(lambda x: 'bace' in str(x.get('task', '')).lower())
    
    for item in tqdm(ds_ref_filtered, desc="Building Index"):
        # 공백 제거 후 키로 사용 (미세한 차이 무시)
        p_text = item.get('prompt_text') or item.get('text', '')
        if p_text:
            ref_lookup[p_text.strip()] = item
            
    print(f"   ✅ Indexed {len(ref_lookup)} unique prompts from Reference.")

    # -------------------------------------------------------
    # 2. Target 데이터 순회하며 매칭 시도
    # -------------------------------------------------------
    print("\n⚖️  Starting Comparison (Matching logic: prompt_text)...")
    
    matched_count = 0
    max_matches_to_show = 3  # 몇 개나 비교하고 멈출지 설정
    
    # 비교할 필드 목록 정의
    fields_to_compare = [
        # (표시 이름, 필드 키, 그래프 내부 여부)
        ('Input Mol Strings', 'input_mol_strings', False), 
        ('Target Text (Label)', 'target_text', False),
        ('Graph Node Feat (x)', 'x', True),
        ('Graph Edge Index', 'edge_index', True),
        ('Graph Edge Attr', 'edge_attr', True),
    ]
    for i, target_item in enumerate(ds_target):
        if matched_count >= max_matches_to_show:
            break
            
        # Target의 prompt_text 가져오기
        tgt_prompt = target_item.get('prompt_text') or target_item.get('text', '')
        if not tgt_prompt:
            continue
            
        tgt_key = tgt_prompt.strip()
        
        # 1. Prompt Text 매칭 확인
        if tgt_key in ref_lookup:
            ref_item = ref_lookup[tgt_key]
            matched_count += 1
            
            print(f"\n{'='*100}")
            print(f"🔥 MATCH FOUND #{matched_count} (Dataset Index: {i})")
            print(f"   Prompt Text: {format_oneline(tgt_prompt)[:100]} ... (Identical)")
            print(f"{'='*100}")
            
            # 2. 세부 항목 비교
            for label, field_key, is_graph in fields_to_compare:
                if is_graph:
                    val_tgt = get_graph_feature(target_item, field_key)
                    val_ref = get_graph_feature(ref_item, field_key)
                else:
                    # input_mol_strings 등은 이름이 조금 다를 수 있어 예외 처리
                    if field_key == 'input_mol_strings':
                         val_tgt = target_item.get('input_mol_strings') or target_item.get('input_mol_string')
                         val_ref = ref_item.get('input_mol_strings') or ref_item.get('input_mol_string')
                    else:
                        val_tgt = target_item.get(field_key)
                        val_ref = ref_item.get(field_key)
                
                # 포맷팅 (한 줄 만들기)
                str_tgt = format_oneline(val_tgt)
                str_ref = format_oneline(val_ref)
                
                print(f"\n🔹 [{label}]")
                print(f"   Target: {str_tgt}")
                print(f"   Writer: {str_ref}")
                
                if str_tgt == str_ref:
                    print("   >> ✅ MATCH")
                else:
                    print("   >> ❌ DIFFERENT")
                    
    if matched_count == 0:
        print("\n⚠️ No exact prompt text matches found.")
        print("   (This is expected if you are comparing the 'Swapped' dataset against the original Reference.)")
        print("   (If comparing original vs original, check if preprocessing differed.)")

if __name__ == "__main__":
    main()

📂 Loading Datasets...
   Ref Size: 58757
   Target Size: 152

🔍 Indexing Reference Dataset by 'prompt_text'...


Building Index: 100%|██████████| 152/152 [00:00<00:00, 1504.12it/s]

   ✅ Indexed 152 unique prompts from Reference.

⚖️  Starting Comparison (Matching logic: prompt_text)...

🔥 MATCH FOUND #1 (Dataset Index: 17)
   Prompt Text: <s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular  ... (Identical)

🔹 [Input Mol Strings]
   Target: <SELFIES> [C][C][N][C][=C][C][=C][Branch2][Ring2][=N][C][=C][Branch2][Ring1][S][C][=Branch1][C][=O][N][C][Branch1][#Branch2][C][C][=C][C][=C][C][=C][Ring1][=Branch1][C@H1][Branch1][C][O][C][N][C][=C][C][=C][C][=C][Ring1][=Branch1][C][=C][Ring2][Ring1][O][Ring2][Ring1][=C][N][Branch1][C][C][S][=Branch1][C][=O][=Branch1][C][=O][C][C][Ring2][Ring2][Ring1] </SELFIES>
   Writer: <SELFIES> [C][C][N][C][=C][C][=C][Branch2][Ring2][=N][C][=C][Branch2][Ring1][S][C][=Branch1][C][=O][N][C][Branch1][#Branch2][C][C][=C][C][=C][C][=C][Ring1][=Branch1][C@H1][Branch1][C][O][C][N][C][=C][C][=C][C][=C][Ring1][=Branch1][C][=C][Ring2][Ring1][O][Ring2][Ring1][=C][N][Branch1][C][C][S][=Branch1][C][=O][=

# ETC

In [8]:
from datasets import load_from_disk
download_bace_blank = load_from_disk('Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_InstructGraph_blank_bace')
len(download_bace_blank)

152

In [9]:
download_bace_blank[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [6, 0, 2, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [6, 0, 3, 5, 2, 0, 1, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 0, 0],
  [7, 0, 1, 5, 0, 0, 1, 0, 0],
  [6, 0, 3, 5, 1, 0, 1, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [7, 0, 2, 5, 0, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1]],
 'edge_index': [[0,
   1,
   1,
 

In [1]:
writer = load_from_disk('Mol-LLM_Custom/checkpoint/mol-llm_testset').filter(lambda x : x['task'] == 'bace')
writer[0].keys()

NameError: name 'load_from_disk' is not defined

In [2]:
writer[0]

NameError: name 'writer' is not defined